In [1]:
from dotenv import load_dotenv
import pandas as pd
load_dotenv()
import os
dataset_path = os.getenv("DATASET_PATH")

In [2]:
df = pd.read_csv(dataset_path)
df.head()

,Title,Authors,Description,Category,Publisher,Price Starting With ($),Publish Date (Month),Publish Date (Year)
0,Goat Brothers,"By Colton, Larry",NaN,"History , General",Doubleday,8.79,January,1993
1,The Missing Person,"By Grumbach, Doris",NaN,"Fiction , General",Putnam Pub Group,4.99,March,1981
2,Don't Eat Your Heart Out Cookbook,"By Piscatella, Joseph C.",NaN,"Cooking , Reference",Workman Pub Co,4.99,September,1983
3,When Your Corporate Umbrella Begins to Leak: A...,"By Davis, Paul D.",NaN,NaN,Natl Pr Books,4.99,April,1991
4,Amy Spangler's Breastfeeding : A Parent's Guide,"By Spangler, Amy",NaN,NaN,Amy Spangler,5.32,February,1997


# EDA

In [3]:
if df.Authors.str.startswith("By ").any():
    df.Authors = df.Authors.str[3:]

df.head()

,Title,Authors,Description,Category,Publisher,Price Starting With ($),Publish Date (Month),Publish Date (Year)
0,Goat Brothers,"Colton, Larry",NaN,"History , General",Doubleday,8.79,January,1993
1,The Missing Person,"Grumbach, Doris",NaN,"Fiction , General",Putnam Pub Group,4.99,March,1981
2,Don't Eat Your Heart Out Cookbook,"Piscatella, Joseph C.",NaN,"Cooking , Reference",Workman Pub Co,4.99,September,1983
3,When Your Corporate Umbrella Begins to Leak: A...,"Davis, Paul D.",NaN,NaN,Natl Pr Books,4.99,April,1991
4,Amy Spangler's Breastfeeding : A Parent's Guide,"Spangler, Amy",NaN,NaN,Amy Spangler,5.32,February,1997


# Vectorize book descriptions

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")


In [5]:
from langchain_chroma import Chroma

# Check if the vector database already exists
db_path = "./chroma_langchain_db"

# Create directory if it doesn't exist
if not os.path.exists(db_path):
    os.makedirs(db_path)

vector_store = Chroma(
    collection_name="books",
    embedding_function=embeddings,
    persist_directory=db_path
)

In [ ]:
from langchain_core.documents import Document
from tqdm import tqdm

# Filter out rows without descriptions since we need text to embed
df_with_desc = df[df.Description.notna()].copy()

# Create Document objects with book descriptions and metadata
documents = []
for _, row in tqdm(df_with_desc.iterrows(), total=len(df_with_desc)):
    doc = Document(
        page_content=f"Title: {row['Title']} - Description: {row['Description']} - Category: {row['Category'] if pd.notna(row['Category']) else 'N/A'}",
        metadata={
            'title': row['Title'],
            'authors': row['Authors'],
            'category': row['Category'] if pd.notna(row['Category']) else '',
            'publisher': row['Publisher'] if pd.notna(row['Publisher']) else '',
            'price': row['Price Starting With ($)'],
            'publish_month': row['Publish Date (Month)'],
            'publish_year': row['Publish Date (Year)']
        }
    )

    # Add documents to the vector store
    vector_store.add_documents([doc])
print(f"Added {len(documents)} books to the vector store")

In [ ]:
# Calculate 20% of the dataframe
sample_size = int(len(df_with_desc) * 0.2)

# Sort by multiple columns to get diverse sample
# Sort by category, year, and price to maximize diversity
df_sorted = df_with_desc.sort_values(
    by=['Category', 'Publish Date (Year)', 'Price Starting With ($)'],
    na_position='first'
)

# Take evenly spaced samples to maximize diversity
step = len(df_sorted) // sample_size
df_sample = df_sorted.iloc[::step][:sample_size]

print(f"Sampling {len(df_sample)} books out of {len(df_with_desc)} (20%)")

# Add sampled documents to vector store
for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    doc = Document(
        page_content=f"Title: {row['Title']} - Description: {row['Description']} - Category: {row['Category'] if pd.notna(row['Category']) else 'N/A'}",
        metadata={
            'title': row['Title'],
            'authors': row['Authors'],
            'category': row['Category'] if pd.notna(row['Category']) else '',
            'publisher': row['Publisher'] if pd.notna(row['Publisher']) else '',
            'price': row['Price Starting With ($)'],
            'publish_month': row['Publish Date (Month)'],
            'publish_year': row['Publish Date (Year)']
        }
    )
    vector_store.add_documents([doc])

print(f"Successfully added {len(df_sample)} diverse books to the vector store")

Sampling 14040 books out of 70204 (20%)


 37%|███▋      | 5126/14040 [25:12<1:17:00,  1.93it/s]

In [10]:
vector_store._collection.count()

302

In [17]:
results = vector_store.similarity_search("galaxy", k=5)

for result in results:
    print('title:',result.metadata['title'])
    print('authors:',result.metadata['authors'])
    print('category:',result.metadata['category'])
    print("-----")

title: Cycles of Fire: Stars, Galaxies and the Wonder of Deep Space
authors: Hartmann, William K.
category: 
-----
title: The Whole Shebang: A State-of-the-Universe(s) Report
authors: Ferris, Timothy
category:  Science , Cosmology
-----
title: Shella
authors: Vachss, Andrew H.
category:  Fiction , Mystery & Detective , General
-----
title: ON THE WING: The Life of Birds: From Feathers to Flight
authors: Brooks, Bruce
category:  Science , General
-----
title: Wolf in the Shadows (A Sharon Mccone Mystery)
authors: Muller, Marcia
category:  Fiction , Mystery & Detective , General
-----
